In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import cPickle
import heapq
from bisect import bisect_left
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.pipeline import Pipeline

In [3]:
data_raw = pd.read_csv('../data/joint_raw.csv', sep='\t', encoding='utf-8')
data_norm = pd.read_csv('../data/joint_norm.csv', sep='\t', encoding='utf-8')

In [4]:
def merge_similar_tags(t):
    if t == u'неисполнение решений суда':
        return u'неисполнение решения суда'
    elif t == u'вывоз детей за границу без разрешения отца':
        return u'вывоз ребёнка за границу без разрешения отца'
    elif t == u'убийство собак':
        return u'убийство собаки'
    elif t == u'ответственность за неисполнения решения суда':
        return u'ответственность за неисполнение решения суда'
    elif t == u'долги по налогам':
        return u'долг по налогам'
    elif t == u'задолженность по жкх':
        return u'задолженность ЖКХ'
    elif t == u'дарственное на квартиру':
        return u'дарственная на квартиру'
    elif t == u'задолженности по кредитам':
        return u'задолженность по кредиту'
    elif t == u'ДТП с пострадавшим':
        return u'ДТП с пострадавшими'
    elif t == u'квартиры по военной ипотеке':
        return u'квартира по военной ипотеке'
    elif t == u'долг судебным приставам':
        return u'долги перед судебными приставами'
    elif t == u'эмансипация несовершеннолетней':
        return u'эмансипация несовершеннолетнего'
    elif t == u'долги по алиментам':
        return u'долг по алиментам'
    elif t == u'выплаты за третьего ребенка':
        return u'выплаты на третьего ребенка'
    elif t == u'документы для продажи квартиры':
        return u'документы при продаже квартиры'
    elif t == u'запрет выезда за границу':
        return u'запреты на выезд за границу'
    elif t == u'отказ от детей':
        return u'отказ от ребенка'
    elif t == u'налог с продажи дома':
        return u'налоги при продаже дома'
    elif t == u'возврат денег за неоказанные услуги':
        return u'возврат денег за неоказанную услугу'
    elif t == u'просрочены права':
        return u'просроченные права'
    elif t == u'займы под материнский капитал':
        return u'займ под материнский капитал'
    elif t == u'права несовершеннолетних детей':
        return u'права несовершеннолетнего ребенка'
    elif t == u'налоговый вычет при ипотеке':
        return u'налоговый вычет по ипотеке'
    elif t == u'протокол административного правонарушения':
        return u'протокол об административном правонарушении'
    elif t == u'отработка при увольнении':
        return u'отработка после увольнения'
    elif t == u'обременение земельного участка':
        return u'обременения земельного участка'
    elif t == u'публичные оскорбления':
        return u'публичное оскорбление'
    elif t == u'долги по исполнительному листу':
        return u'долги по исполнительным листам'
    elif t == u'удержание по исполнительному листу':
        return u'удержание по исполнительным листам'
    elif t == u'пропущены сроки вступления в наследство':
        return u'пропущенный срок вступления в наследство'
    elif t == u'несанкционированные свалки':
        return u'несанкционированная свалка'
    elif t == u'кража из магазина':
        return u'кража в магазине'
    elif t == u'нарушения авторского права':
        return u'нарушение авторского права'
    else:
        return t

# data_raw.loc[:,'tag'] = data_raw.tag.apply(merge_similar_tags)

In [5]:
X = data_norm.text.values
y = data_norm.tag.values
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [6]:
tags_raw = data_raw.tag.unique()
tags_norm = data_norm.tag.unique()

In [7]:
# cPickle.dump(tags_raw, open('../data/tags_raw.p', 'wb'))

In [8]:
print len(tags_raw) - len(tags_norm)
for t in tags_norm:
    tt = data_raw[data_norm.tag == t].tag.unique()
    if len(tt) > 1:
        for _t in tt:
            print _t

3
возврат долга по расписке
возврат долга без расписки
договор аренды транспортного средства без экипажа
договор аренды транспортного средства с экипажем
сбил пешехода на пешеходном переходе
сбил пешехода вне пешеходного перехода


In [9]:
# data_raw.to_csv('../data/joint_raw.csv', sep='\t', encoding='utf-8', index=False)

In [10]:
tags_norm_to_raw = []
for t in tags_norm:
    if t == u'возврат долг расписка':
        tags_norm_to_raw.append([u'возврат долга по расписке', u'возврат долга без расписки'])
    elif t == u'договор аренда транспортный средство экипаж':
        tags_norm_to_raw.append([u'договор аренды транспортного средства без экипажа',
                u'договор аренды транспортного средства с экипажем'])
    elif t == u'сбить пешеход пешеходный переход':
        tags_norm_to_raw.append([u'сбил пешехода на пешеходном переходе', 
                                 u'сбил пешехода вне пешеходного перехода'])
    else:
        tags_norm_to_raw.append(data_raw.tag[data_norm.tag == t].values[0])

In [11]:
# cPickle.dump(tags_norm, open('../data/tags_norm.p', 'wb'))
# cPickle.dump(tags_norm_to_raw, open('../data/tags_norm_to_raw.p', 'wb'))

In [12]:
import pymorphy2
from nltk.tokenize import RegexpTokenizer

morph = pymorphy2.MorphAnalyzer()
tokenizer = RegexpTokenizer('\w+')

def normalize(s):
    return ' '.join(map(lambda w: morph.parse(w)[0].normal_form, tokenizer.tokenize(s)))

def get_normalized_tokens(s):
    return map(lambda w: morph.parse(w)[0].normal_form, tokenizer.tokenize(s))

def remove_minor_pos(s):
    return ' '.join(filter(lambda w: morph.parse(w)[0].tag.POS not in ['PREP', 'CONJ', 'PRCL', 'INTJ'], tokenizer.tokenize(s)))

---

Синонимы

In [73]:
%%time
tags_synonyms = []
with open('../data/syn.txt') as f:
    lines = f.readlines()
    lines = map(lambda l: l.decode('utf-8'), lines)
    lines.sort()
    for tag in tags_norm:
        tag_syns = []
        for word in tag.split(' '):
            idx = bisect_left(lines, word+'|')
            line = lines[idx]
            if line.startswith(word+'|'):
                tag_syns += get_normalized_tokens(line.split('|')[1])
        tags_synonyms.append(tag_syns)

CPU times: user 25.8 s, sys: 112 ms, total: 25.9 s
Wall time: 25.7 s


In [74]:
cPickle.dump(tags_synonyms, open('../data/tags_synonyms.p', 'wb'))

In [75]:
tags_synonyms = cPickle.load(open('../data/tags_synonyms.p', 'rb'))

---

Неудачные веса

In [70]:
%%time
for tag_idx in xrange(len(tags_norm)):
    tag = tags_norm[tag_idx]
    for word_idx in xrange(len(tag)):
        w1 = np.array(map(lambda text: w in text, X_train[y_train == t])).sum() / float((y_train == t).sum())
        w2 = np.array(map(lambda text: w in text, X_train[y_train != t])).sum() / float((y_train != t).sum())
        tag[word_idx] = (tag[word_idx], w1/w2)

CPU times: user 21min 19s, sys: 1.44 s, total: 21min 21s
Wall time: 21min 17s


In [ ]:
def predict_n_tags(text, n):
    predicted_n = [(-1, -1)] * n
    heapq.heapify(predicted_n)
    for tag in tags_norm:
        trust_level = 0.
        for word, weight in tag:
            if word in text:
                trust_level += weight
        heapq.heappushpop(predicted_n, (trust_level, ' '.join(zip(*tag)[0])))
    return predicted_n

In [ ]:
idx = 0
print y_train[idx]
print X_train[idx]
predicted_3 = predict_n_tags(X_train[idx], 3)
for pair in predicted_3:
    print u'{0} - {1}'.format(pair[1], pair[0])

---

In [13]:
# Counts the number of words from <words>, that appear in <text>
def count_entries(words, text):
    count = 0
    for word in words:
        if word in text:
            count += 0.5
    return count

PAIR_WEIGHT = 2
def count_entries2(words, text):
    count = 0
    words = words.split(' ')
    for i in range(len(words)-1):
        if words[i] in text:
            count += 1
            pair = ' '.join((words[i], words[i+1]))
            if pair in text:
                count += PAIR_WEIGHT
    if words[-1] in text:
        count += 1
    return count
def count_part_entries2(words, text, part_size, weight, pair_weight):
    count = 0
    words = words.split(' ')
    words = map(lambda w: w[:part_size], words)
    text = ' '.join(map(lambda w: w[:part_size], text.split(' ')))
    for i in range(len(words)-1):
        if words[i] in text:
            count += weight
            pair = ' '.join((words[i], words[i+1]))
            if pair in text:
                count += pair_weight * weight
    if words[-1] in text:
        count += 1
    return count

In [14]:
# Returns <num_tags> the most suitable tags for <text>
def get_top_tags(text, tags, num_tags, part_size, two_counts, part_weight=1, pair_weight=0):
    similarities = []
    for tag in tags:
        if two_counts:
            similarities.append(count_entries2(tag, text) / float(len(tag.split(' '))))
            similarities[-1] += count_part_entries2(tag, text, part_size, part_weight, pair_weight) / float(len(tag.split(' ')))
        else:
            similarities.append(count_part_entries2(tag, text, part_size, part_weight, pair_weight) / float(len(tag.split(' '))))
#         similarities[-1] += count_part_entries2(tag, text, 5) / float(len(tag.split(PAIR_WEIGHT ' ')))
#     for tag, syns in zip(tags, tags_synonyms):
#         similarities.append(count_entries2(tag, text) / float(len(tag)))
#         similarities[-1] += count_entries(syns, text) / float(len(tag))
    result = []
    for _ in range(num_tags):
        idx = np.argmax(similarities)
        result.append((tags[idx], similarities[idx]))
        similarities[idx] = 0.
    return result

In [125]:
%%time

_, X_small, _, y_small = train_test_split(X, y, test_size=0.2)
    
num_tested = len(X_small)
for part_size in (4,5,6):
    for two_counts in (False,):
        if two_counts:
            for part_weight in (0.5, 1., 1.5):
                num_guessed = 0
                for i in xrange(num_tested):
                    top3 = get_top_tags(X_small[i], tags_norm, 3, part_size, two_counts, part_weight)
                    if y_small[i] in zip(*top3)[0]:
                        num_guessed += 1
                acc = num_guessed / float(num_tested)
                print part_size, two_counts, part_weight
                print acc
        else:
            for pair_weight in (0,1,2):
                num_guessed = 0
                for i in xrange(num_tested):
                    top3 = get_top_tags(X_small[i], tags_norm, 3, part_size, two_counts)
                    if y_small[i] in zip(*top3)[0]:
                        num_guessed += 1
                acc = num_guessed / float(num_tested)
                print part_size, two_counts, pair_weight
                print acc

4 False 0
0.314354496647
4 False 1
0.314354496647
4 False 2
0.314354496647
5 False 0
0.380786793787
5 False 1
0.380786793787
5 False 2
0.380786793787
6 False 0
0.409063986702
6 False 1
0.409063986702
6 False 2
0.409063986702
CPU times: user 4h 26min 3s, sys: 27.5 s, total: 4h 26min 31s
Wall time: 4h 25min 34s


In [21]:
%%time
                
# X_small = X
# y_small = y
accuracies = []
for j in range(3):
    _, X_small, _, y_small = train_test_split(X, y, test_size=0.01)
    num_guessed = 0
    num_tested = len(X_small)
    for i in xrange(num_tested):
        top3 = get_top_tags(X_small[i], tags_norm, 3, 6, False, 1, 1)
        if y_small[i] in zip(*top3)[0]:
            num_guessed += 1
    acc = num_guessed / float(num_tested)
    print acc
    accuracies.append(acc)

print 'Avg. accuracy: {0}'.format(np.mean(accuracies))

0.481467329003
0.489491784486
0.510126098586
Avg. accuracy: 0.493695070692
CPU times: user 4min 27s, sys: 456 ms, total: 4min 27s
Wall time: 4min 26s


---

In [ ]:
tags_sorted = data_tagged.tag.value_counts().index.values
tag_encoder = dict(zip(tags_sorted, range(len(tags_sorted))))
y_tag_encoded = np.array(map(lambda c: tag_encoder[c], y_tag))
y_tag_encoded[:5]

TypeError: If no scoring is specified, the estimator passed should have a 'score' method. The estimator Pipeline(steps=[('vect', TfidfVectorizer(analyzer=u'word', binary=True, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=False, max_df=0.7, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=True...s=50, n_estimators=10, n_neighbors=5,
     radius=1.0, radius_cutoff_ratio=0.9, random_state=None))]) does not.

---

Генерим фичи из тегов

In [21]:
key_words = []
for tag in tags_norm:
    key_words += tag
key_words = set(key_words)

In [22]:
%%time

X_small = X
y_small = y
# _, X_small, _, y_small = train_test_split(X, y, test_size=0.1)
pipeline = Pipeline([
    ('vect', CountVectorizer(binary=True, lowercase=False, vocabulary=key_words)),
    ('clf', SGDClassifier())
#     ('clf', LogisticRegression(solver='lbfgs', multi_class='multinomial'))
])
cross_val_score(pipeline, X_small, y_small, verbose=4, scoring='accuracy').mean()

[CV] no parameters to be set .........................................
[CV] ................ no parameters to be set, score=0.000284 - 1.8min
[CV] no parameters to be set .........................................


KeyboardInterrupt: 

In [139]:
# %%time

from sklearn.neighbors import LSHForest
from sklearn.metrics import accuracy_score


key_words = []
for tag in tags_norm:
    key_words += tag
key_words = set(key_words)    

# _, X_small, _, y_small = train_test_split(X, y, test_size=0.1)
# pipeline = Pipeline([
#     ('vect', CountVectorizer(binary=True, lowercase=False, vocabulary=key_words)),
#     ('clf', LSHForest())
# #     ('clf', LogisticRegression(solver='lbfgs', multi_class='multinomial'))
# ])
# cross_val_score(pipeline, X_small, y_small, verbose=4, scoring='accuracy').mean()


accuracies = []
for j in range(3):
    _, X_small, _, y_small = train_test_split(X, y, test_size=0.5)
    X_small = CountVectorizer(binary=True, lowercase=False, vocabulary=key_words).fit_transform(X_small)
    lshf = LSHForest().fit(X_small, y_small)
    num_guessed = 0
    num_tested = X_small.shape[0]
    for i in xrange(num_tested):
        if i == lshf.kneighbors(X_small[i], 1)[1][0]:
            num_guessed += 1
    acc = num_guessed / float(num_tested)
    print acc
    accuracies.append(acc)

print 'Avg. accuracy: {0}'.format(np.mean(accuracies))

KeyboardInterrupt: 

In [144]:
import re

class Porter:
	PERFECTIVEGROUND =  re.compile(u"((ив|ивши|ившись|ыв|ывши|ывшись)|((?<=[ая])(в|вши|вшись)))$")
	REFLEXIVE = re.compile(u"(с[яь])$")
	ADJECTIVE = re.compile(u"(ее|ие|ые|ое|ими|ыми|ей|ий|ый|ой|ем|им|ым|ом|его|ого|ему|ому|их|ых|ую|юю|ая|яя|ою|ею)$")
	PARTICIPLE = re.compile(u"((ивш|ывш|ующ)|((?<=[ая])(ем|нн|вш|ющ|щ)))$")
	VERB = re.compile(u"((ила|ыла|ена|ейте|уйте|ите|или|ыли|ей|уй|ил|ыл|им|ым|ен|ило|ыло|ено|ят|ует|уют|ит|ыт|ены|ить|ыть|ишь|ую|ю)|((?<=[ая])(ла|на|ете|йте|ли|й|л|ем|н|ло|но|ет|ют|ны|ть|ешь|нно)))$")
	NOUN = re.compile(u"(а|ев|ов|ие|ье|е|иями|ями|ами|еи|ии|и|ией|ей|ой|ий|й|иям|ям|ием|ем|ам|ом|о|у|ах|иях|ях|ы|ь|ию|ью|ю|ия|ья|я)$")
	RVRE = re.compile(u"^(.*?[аеиоуыэюя])(.*)$")
	DERIVATIONAL = re.compile(u".*[^аеиоуыэюя]+[аеиоуыэюя].*ость?$")
	DER = re.compile(u"ость?$")
	SUPERLATIVE = re.compile(u"(ейше|ейш)$")
	I = re.compile(u"и$")
	P = re.compile(u"ь$")
	NN = re.compile(u"нн$")

	def stem(word):
		word = word.lower()
		word = word.replace(u'ё', u'е')
		m = re.match(Porter.RVRE, word)
		if m.groups():
			pre = m.group(1)
			rv = m.group(2)
			temp = Porter.PERFECTIVEGROUND.sub('', rv, 1)
			if temp == rv:
				rv = Porter.REFLEXIVE.sub('', rv, 1)
				temp = Porter.ADJECTIVE.sub('', rv, 1)
				if temp != rv:
					rv = temp
					rv = Porter.PARTICIPLE.sub('', rv, 1)
				else:
					temp = Porter.VERB.sub('', rv, 1)
					if temp == rv:
						rv = Porter.NOUN.sub('', rv, 1)
					else:
						rv = temp
			else:
				rv = temp
			
			rv = Porter.I.sub('', rv, 1)

			if re.match(Porter.DERIVATIONAL, rv):
				rv = Porter.DER.sub('', rv, 1)

			temp = Porter.P.sub('', rv, 1)
			if temp == rv:
				rv = Porter.SUPERLATIVE.sub('', rv, 1)
				rv = Porter.NN.sub(u'н', rv, 1)
			else:
				rv = temp
			word = pre+rv
		return word
	stem=staticmethod(stem)
	
print Porter.stem(u'покупки')

кредитов
